In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score



In [2]:
# Load the dataset
df = pd.read_csv('../data/processed/cleaned_data_Namrata.csv')


In [3]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('/Users/ayushyapare/Desktop/Ayushyas_Life/Work/Projects/Snippets')

from DataFrame_Analysis import analyze_dataframe

In [4]:
# Perform EDA now
#analyze_dataframe(df)

In [5]:
df.columns

Index(['sq_mt_built', 'n_rooms', 'n_bathrooms', 'buy_price',
       'buy_price_by_area', 'has_central_heating', 'has_individual_heating',
       'has_ac', 'has_fitted_wardrobes', 'has_lift', 'is_exterior',
       'energy_certificate', 'has_parking', 'neighborhood', 'District',
       'HouseType'],
      dtype='object')

In [6]:
# Replace '8.0+' with '8'
#df['n_rooms'] = df['n_rooms'].str.replace('8.0+', '8', regex=False)
#df['n_bathrooms'] = df['n_bathrooms'].str.replace('8.0+', '8', regex=False)

In [7]:
# Assuming the dataset is already loaded into df
# Define features and target variable
X = df[['sq_mt_built', 'n_rooms', 'n_bathrooms', 'buy_price',
        'has_central_heating', 'has_individual_heating',
       'has_ac', 'has_fitted_wardrobes', 'has_lift', 'is_exterior',
       'energy_certificate', 'has_parking', 'neighborhood', 'District',
       'HouseType']]
y = df['buy_price_by_area']



In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identify numerical and categorical columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Define preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with the mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

# Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [9]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet


In [11]:
# Initialize models
logistic_regression_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

ridge_classifier_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RidgeClassifier())
])

lasso_logistic_regression_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(penalty='l1', solver='saga'))
])

elastic_net_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', ElasticNet())
])

# Train and evaluate


In [12]:
#conda install -c conda-forge pycaret

In [13]:
from pycaret.classification import *

In [14]:
# Check the class distribution of the target variable
class_counts = y_train.value_counts()
print(class_counts)

# Identify classes with only one instance
single_instance_classes = class_counts[class_counts == 1].index

# Filter out rows in X_train and y_train where the target variable is in the single_instance_classes
mask = ~y_train.isin(single_instance_classes)
X_train_filtered = X_train[mask]
y_train_filtered = y_train[mask]

# Check the class distribution again to ensure single instance classes are removed
class_counts_filtered = y_train_filtered.value_counts()
print(class_counts_filtered)

buy_price_by_area
5000    120
2000     76
3000     75
2500     74
4000     70
       ... 
7842      1
3972      1
8822      1
2548      1
8958      1
Name: count, Length: 5273, dtype: int64
buy_price_by_area
5000    120
2000     76
3000     75
2500     74
4000     70
       ... 
5468      2
4522      2
2280      2
1528      2
4404      2
Name: count, Length: 3429, dtype: int64


In [15]:
# Setup the environment in PyCaret
regression_setup = setup(
    data=pd.concat([X_train_filtered, y_train_filtered], axis=1),
    target = 'buy_price_by_area',
    session_id=9,
    #max_encoding_ohe=600, # columns with 600 or less categories will be One-hot encoded ELSE target encoding
    #rare_to_value=0.008, # Categories with less than 0.008 (0.8%) of the data will be grouped into a new category (Other)
    #rare_value='Other',
    #fix_imbalance = True,
    #fix_imbalance_method = 'SMOTE',
    transformation = True,
    transformation_method = 'yeo-johnson',
    #experiment_name='Clsfctn_tel_cust_ayushya_(dm)',
    log_experiment = False,
    normalize=True,  # True, False
    normalize_method='zscore',  # 'zscore', 'minmax', 'maxabs', 'robust'
    n_jobs=-1)

In [ ]:
# Log the parameters and metrics with MLflow
best_model = compare_models()


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:30:18
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/61 [00:00<?, ?it/s]